In [ ]:
"""
Author: Ziyu He 184497359@qq.com
Xuanji Xiao superwander@outlook.com
"""

%tensorflow_version 1.x
import random
import numpy as np
from tqdm import tqdm
import json
import pandas as pd
import numpy as np
import tensorflow as tf
root_path = "/Data/"
tf.__version__

In [ ]:
movie_df = pd.read_csv(root_path + "movies.dat", sep="::", header = None)
rating_df = pd.read_csv(root_path + "ratings.dat", sep="::", header = None)
user_df = pd.read_csv(root_path + "users.dat", sep="::", header = None)

movie_df.columns = ['movie_id','title','genres']
rating_df.columns = ['user_id','movie_id','rating','timestamp']
user_df.columns = ['user_id','gender','age','occupation','zip']

movie_df.genres = movie_df.genres.apply(lambda x: x.split("|"))
merged_df = pd.merge(pd.merge(rating_df,movie_df),user_df)

In [ ]:
# Every user has watched at least 20 movies, can't simulate cold start, we
# randomly select 1/10 history interaction data to similate cold start problem
merged_df = merged_df.sample(frac = 0.1) 
hist_len = merged_df.user_id.value_counts()
print("total user: {}".format(len(hist_len)))
# sparse user dara: every user has watched less than 30 movies
sparse_hist_len = hist_len[hist_len < 30]
print("sparse user: {}".format(len(sparse_hist_len)))
print(hist_len)
print("total_interactions: {}".format(sum(hist_len.values)))
print("total_interactions_sparse: {}".format(sum(sparse_hist_len.values)))
print("=============")
print(sparse_hist_len)

In [ ]:
#select sparse data
merged_df = merged_df[merged_df.user_id.isin(sparse_hist_len.index)]

In [ ]:
#make sure every user has watched at least 3 movies
valid_user = merged_df.user_id.value_counts()[merged_df.user_id.value_counts() > 2].index
merged_df['valid'] = merged_df.user_id.apply(lambda x: x in valid_user)
merged_df = merged_df[merged_df.valid]
merged_df.drop(['valid'], axis = 1, inplace = True)

In [ ]:
merged_df.sort_values("timestamp", inplace=True)

In [ ]:
#search the static neighbours with the same attributes 【'gender', 'age', 'occupation'】
def find_neighbour(uid, user_df):
    select_ = user_df
    user = select_[select_.user_id == uid]
    for feature in ['gender', 'age', 'occupation']:
        select_ = select_[select_[feature] == user[feature].values[0]]
    return select_

In [ ]:
#the interest tag of the users 
from collections import Counter
interest_dict = dict()
for u, hist in merged_df.groupby('user_id'):
    concat_ = np.concatenate(hist.genres.tolist())
    interest_dict[u] = [x[0] for x in Counter(concat_).most_common(3)]

In [ ]:
#the users with the top 3 most common tags
interest_dict
def search_tag_neig(uid, id_tags, new):
    ans = []
    if new:
        for i in id_tags:
            if id_tags[i] == id_tags[uid]:
                ans.append(i)
    return ans

In [ ]:
movie_df['genres_str'] = movie_df.genres.apply(lambda x:str(x))

In [ ]:
train_movie_df = movie_df[movie_df.movie_id.isin(train_merged_df.movie_id.unique())]

In [ ]:
movie_tag_dict = {}
for i in range(train_movie_df.shape[0]):
    if train_movie_df.iloc[i].genres_str in movie_tag_dict:
        movie_tag_dict[train_movie_df.iloc[i].genres_str].append(train_movie_df.iloc[i].movie_id)
    else:
        movie_tag_dict[train_movie_df.iloc[i].genres_str] = [train_movie_df.iloc[i].movie_id]

In [ ]:
movie_tag_dict

In [ ]:
movie_df.iloc[1].genres_str

In [ ]:
#item with the same tag
def search_same_tag_movie(itemid, df):
    target_item = df[df.movie_id == itemid]
    target_genres = target_item.genres_str.unique()
    return_list = []
    for i in range(df.shape[0]):
        row = df.iloc[i]
        movie_id = row.movie_id
        genres = row.genres_str
        #print(target_genres)
        #print(genres)
        if genres == target_genres:
            return_list.append(movie_id)
    return return_list

same_tag_movie_dict = {}
for i in tqdm(range(train_movie_df.shape[0])):
    movie_id_ = train_movie_df.iloc[i]
    same_tag_movie_dict[movie_id_.movie_id] = movie_tag_dict[movie_id_.genres_str]

In [ ]:
# users with same attributes [gender、age、occpuation]
tag_friends = []
for i in merged_df.user_id.unique():
    tag_friends.append(search_tag_neig(i, interest_dict, True))

In [ ]:
tag_dict = {
"Drama":"0",
"Comedy":"1",
"Action":"2",
"Thriller":"3",
"Romance":"4",
"Horror":"5",
"Adventure":"6",
"Sci-Fi":"7",
"Children's":"8",
"Crime":"9",
"War":"10",
"Documentary":"11",
"Musical":"12",
"Mystery":"13",
"Animation":"14",
"Fantasy":"15",
"Western":"16",
"Film-Noir":"17"
}

In [ ]:
#delete the test dataset info to prevent data-leakage
test_index = []
for u_id, hist in merged_df.groupby('user_id'):
    test_index.append(hist.iloc[-1].name)
train_merged_df = merged_df[~merged_df.index.isin(test_index)]

#create a itemcf user_item matrix
max_item = max(train_merged_df.movie_id.unique() + 1)
max_user = max(train_merged_df.user_id.unique() + 1)
co_matrix = np.zeros((max_item,max_user))

print(co_matrix.shape)
for row_num in range(train_merged_df.shape[0]):
    row = train_merged_df.iloc[row_num]
    co_matrix[row.movie_id, row.user_id] = row.rating

def cos_value(a, b):
    return np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))

movie_id_list = train_merged_df.movie_id.value_counts().index
simi_dict = {}
for item in tqdm(movie_id_list):
    temp_list = []
    for other_item in movie_id_list:
        temp_list.append((other_item, cos_value(co_matrix[item], co_matrix[other_item])))
    temp_list = sorted(temp_list, key = lambda x:x[1], reverse = True)
    temp_list = [x[0] for x in temp_list[:10]]
    simi_dict[item] = temp_list

In [ ]:
len(same_tag_movie_dict)

In [ ]:
"""
preprocess the dataset
"""
def gen_data_set(data, negsample):

    data.sort_values("timestamp", inplace=True)
    item_ids = data['movie_id'].unique()
    train_set = []
    test_set = []
    for reviewerID, hist in tqdm(data.groupby('user_id')):
        pos_list = hist['movie_id'].tolist()
        rating_list = hist['rating'].tolist()
        genres_list = hist['genres'].tolist()

        #负采样negsample个
        if negsample > 0:
            candidate_set = list(set(item_ids) - set(pos_list))
            neg_list = np.random.choice(candidate_set,size=len(pos_list)*negsample,replace=True)
        
        #the neighbours with the same characteristics(gender\age\occupation)
        neighbours = find_neighbour(reviewerID, user_df).user_id.values
        
        #the neighbours with the same tags(Comedy\Children\Drama\...)
        tag_neighbours = search_tag_neig(reviewerID, interest_dict, True)
        
        for i in range(1, len(pos_list)):
            hist = pos_list[:i]
            
            #the similar movie of the predicted movie (except: the movie only exists in test data)
            try:
                total_similar = simi_dict[pos_list[i]]
            except:
                total_similar = []

            # movies with the same tag of the predicted movie
            #tag_similar_movie = search_same_tag_movie(pos_list[i], movie_df)
            try:
                tag_similar_movie = same_tag_movie_dict[pos_list[i]]
            except:
                print(pos_list[i])
                tag_similar_movie = []
            
            #the similar movies of all history movies (except: the movie only exists in test data)
            hist_similar = []
            for hist_movie in hist:
                try:
                    hist_similar = hist_similar + simi_dict[hist_movie]
                except:
                    hist_similar = hist_similar + []
            
            #the tag_info of the predicted vedio
            vedio_tag = [tag_dict[x] for x in genres_list[i]]
            
            #the tag_info of all history movies
            tags = np.concatenate(genres_list[:i])
            tags_ = [tag_dict[x] for x in tags]
            
            if i != len(pos_list) - 1:
                train_set.append((reviewerID, hist, pos_list[i], 1, len(hist[::-1]), neighbours, tag_neighbours, tags_, vedio_tag, total_similar, hist_similar, tag_similar_movie))
                for negi in range(negsample):
                    train_set.append((reviewerID, hist, neg_list[i*negsample+negi], 0,len(hist[::-1]),neighbours, tag_neighbours, tags_, vedio_tag, total_similar, hist_similar, tag_similar_movie))
            else:
                test_set.append((reviewerID, hist, pos_list[i], 1,len(hist[::-1]),neighbours, tag_neighbours, tags_, vedio_tag, total_similar, hist_similar, tag_similar_movie))
                for negi in range(negsample):
                    test_set.append((reviewerID, hist, neg_list[i*negsample+negi], 0,len(hist[::-1]),neighbours, tag_neighbours, tags_, vedio_tag, total_similar, hist_similar, tag_similar_movie))

    random.shuffle(train_set)
    random.shuffle(test_set)

    print(len(train_set[0]),len(test_set[0]))
    return train_set,test_set

def gen_model_input(train_set,user_profile):

    train_uid = np.array([line[0] for line in train_set])
    train_seq = [line[1] for line in train_set]
    train_iid = np.array([line[2] for line in train_set])
    train_label = np.array([line[3] for line in train_set])
    train_hist_len = np.array([line[4] for line in train_set])
    train_tag_neighbours = np.array([line[5] for line in train_set])
    train_neighbours = np.array([line[6] for line in train_set])
    train_tags = np.array([line[7] for line in train_set])
    movie_tags = np.array([line[8] for line in train_set])
    similari_movies = np.array([line[9] for line in train_set])
    hist_similar = np.array([line[10] for line in train_set])
    tag_similar_movie = np.array([line[11] for line in train_set])
    
    train_model_input = {"label": train_label, "user_id": train_uid, "movie_id": train_iid,
                         "hist_movie_id": train_seq, "hist_len": train_hist_len, "neighbours": train_neighbours, 
                         "tags":train_tags, "movie_tag": movie_tags, "similar_movies":similari_movies, 
                         "history_similar_movies":hist_similar, "tag_neighbours": train_tag_neighbours, "tag_similar_movie":tag_similar_movie}

    for key in ["gender", "age", "occupation"]:
        #transfer the uid to the index in user_df
        train_model_input[key] = user_profile.iloc[train_model_input['user_id'] - 1][key].values

    return train_model_input

                                             #3 negative samples 
train_set, test_set = gen_data_set(merged_df, 3)
train_dataset = gen_model_input(train_set, user_df)
test_dataset = gen_model_input(test_set, user_df)

print(train_dataset.keys())

class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        if isinstance(obj, np.integer):
            return int(obj)
        return json.JSONEncoder.default(self, obj)

with open(root_path + 'train_data.json', 'w') as outfile:
    json.dump(train_dataset, outfile, cls=NumpyEncoder)
    
with open(root_path + 'test_data.json', 'w') as outfile:
    json.dump(test_dataset, outfile, cls=NumpyEncoder)

data_fea = list(train_dataset.keys())
print(train_dataset.keys())

In [ ]:
"""
transfer to tfrecord
"""

class handle_data():
    
    def __init__(self, dataset):
        self.dataset = dataset
        self.data_fea = list(dataset.keys())
        self._get_next_data = self._get_single_data()
    
    def _get_single_data(self):
        data_len = len(self.dataset['user_id'])
        data_dict = dict()
        i = 0
        while i < data_len:
            for fea in self.data_fea:
                data_dict[fea] = self.dataset[fea][i]
            yield data_dict
            i += 1
          
    @property
    def get_next_data(self):
        return self._get_next_data
    
    @property
    def create_next_example(self):
        """
        'label', 'user_id', 'movie_id', 'hist_movie_id', 'hist_len', 
        'neighbours', 'tags', 'movie_tag', 'similar_movies', 'history_similar_movies',
        'tag_neighbours', 'gender', 'age', 'occupation'
        """
        
        next_ = next(self.get_next_data)
        example = tf.train.Example(features=tf.train.Features(feature={
            
            'user_id': tf.train.Feature(
                bytes_list=tf.train.BytesList(
                    value=[str(next_['user_id']).encode('utf-8')])),        
            
            'user_id_offline': tf.train.Feature(
                bytes_list=tf.train.BytesList(
                    value=[str(next_['user_id']).encode('utf-8')])),
            
            'user_id_offline_fixed': tf.train.Feature(
                bytes_list=tf.train.BytesList(
                    value=[str(next_['user_id']).encode('utf-8')])),
            
            'movie_id': tf.train.Feature(
                bytes_list=tf.train.BytesList(
                    value=[str(next_['movie_id']).encode('utf-8')])),
            
            # multi-value feature
            'hist_movie_id': tf.train.Feature(
                bytes_list=tf.train.BytesList(
                    value=[str(m).encode('utf-8') for m in next_['hist_movie_id']])),
            
            # multi-value feature
            'neighbours': tf.train.Feature(
                bytes_list=tf.train.BytesList(
                    value=[str(m).encode('utf-8') for m in next_['neighbours']])),
            
            # multi-value feature
            'tag_neighbours': tf.train.Feature(
                bytes_list=tf.train.BytesList(
                    value=[str(m).encode('utf-8') for m in next_['tag_neighbours']])),
            
            # multi-value feature
            'user_tags': tf.train.Feature(
                bytes_list=tf.train.BytesList(
                    value=[str(m).encode('utf-8') for m in next_['tags']])),
            
            # multi-value feature
            'movie_tags': tf.train.Feature(
                bytes_list=tf.train.BytesList(
                    value=[str(m).encode('utf-8') for m in next_['movie_tag']])),
            
            # multi-value feature
            'similar_movies': tf.train.Feature(
                bytes_list=tf.train.BytesList(
                    value=[str(m).encode('utf-8') for m in next_['similar_movies']])),
            
            # multi-value feature
            'history_similar_movies': tf.train.Feature(
                bytes_list=tf.train.BytesList(
                    value=[str(m).encode('utf-8') for m in next_['history_similar_movies']])),

            # multi-value feature
            'tag_similar_movie': tf.train.Feature(
                bytes_list=tf.train.BytesList(
                    value=[str(m).encode('utf-8') for m in next_['tag_similar_movie']])),
            
            'hist_len': tf.train.Feature(
                bytes_list=tf.train.BytesList(
                    value=[str(next_['hist_len']).encode('utf-8')])),
            
            'gender': tf.train.Feature(
                bytes_list=tf.train.BytesList(
                    value=[str(next_['gender']).encode('utf-8')])),
            
            'age' : tf.train.Feature(
                bytes_list=tf.train.BytesList(
                    value=[str(next_['age']).encode('utf-8')])),
            
            'occupation': tf.train.Feature(
                bytes_list=tf.train.BytesList(
                    value=[str(next_['occupation']).encode('utf-8')])),
            
            'label': tf.train.Feature(
                bytes_list=tf.train.BytesList(
                    value=[str(next_['label']).encode('utf-8')])),
            
        }))
        return example

train_data_handler = handle_data(train_dataset)
test_data_handler = handle_data(test_dataset)

with tf.io.TFRecordWriter(root_path + 'train.tfrecord') as writer:
    while True:
        try:
            writer.write(train_data_handler.create_next_example.SerializeToString())
        except:
            break
            
with tf.io.TFRecordWriter(root_path + 'test.tfrecord') as writer:
    while True:
        try:
            writer.write(test_data_handler.create_next_example.SerializeToString())
        except:
            break

In [ ]:
def _join_pipeline(map_functions):

    def joined_map_fn(example):
        for map_fn in map_functions:
            example = map_fn(example)
        return example

    return joined_map_fn

def _dataset_map(dataset, map_functions):
    if len(map_functions) > 0:
        map_fn = _join_pipeline(map_functions)
        dataset = dataset.map(map_fn)
    return dataset

def _transform(example):
    read_features = dict()
    for key in ['user_tags', 'movie_tags', 'user_id_offline', 'user_id_offline_fixed', 'age', 'gender', 'hist_movie_id', 'label', 'movie_id', 'neighbours', 'occupation', 'user_id', 'similar_movies', 'history_similar_movies', 'tag_neighbours','tag_similar_movie']:
        if key == 'hist_movie_id' or key == 'neighbours' or key == 'user_tags' or key == 'movie_tags' or key == 'similar_movies' or key == 'history_similar_movies' or key == 'tag_neighbours' or key == 'tag_similar_movie':
            read_features[key] = tf.FixedLenSequenceFeature([],dtype=tf.string,allow_missing=True)
        else:
            read_features[key] = tf.FixedLenFeature([], dtype=tf.string)

    # Extract features from serialized data
    read_data = tf.parse_example(serialized=example,
                                        features=read_features)

    for k, v in read_data.items():
        read_data[k] = tf.identity(v, name=k)
    return read_data

In [ ]:
tf.compat.v1.set_random_seed(1)

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf

tf.reset_default_graph()
import os
import time

# create the dataset
dataset = tf.data.TFRecordDataset('train.tfrecord')
dataset = dataset.batch(1000, True)
dataset = _dataset_map(dataset, [_transform])
iterator = dataset.make_initializable_iterator()

test_dataset = tf.data.TFRecordDataset('test.tfrecord')
test_dataset = test_dataset.batch(1000, True)
test_dataset = _dataset_map(test_dataset, [_transform])
test_iterator = test_dataset.make_initializable_iterator()

In [ ]:
class NumericalStat(object):

    def __init__(self):
        self._min = float("inf")
        self._max = float("-inf")
        self._sum = 0.0
        self._square_sum = 0.0
        self._count = 0

    def update(self, values):
        assert np.issubdtype(values.dtype, np.number)
        self._min = min(np.min(values), self._min)
        self._max = max(np.max(values), self._max)
        self._sum += np.sum(values)
        self._square_sum += np.sum(values * values)
        self._count += values.size

    @property
    def min(self):
        return self._min

    @property
    def max(self):
        return self._max

    @property
    def mean(self):
        return self._sum / float(self._count)

    @property
    def std(self):
        return (self._square_sum / float(self._count) - self.mean * self.mean) ** 0.5

    @property
    def n_samples(self):
        return self._count

    def __repr__(self):
        return "stat(type=numerical, min=%f, max=%f, mean=%f, std=%f, n=%d)" % (
            self.min,
            self.max,
            self.mean,
            self.std,
            self.n_samples,
        )


class CategoricalStat(object):

    def __init__(self):
        self._values = defaultdict(int)
        self._count = 0

    def update(self, values):
        for value in values.flatten():
            self._values[value] += 1
        self._count += values.size

    def values_top_k(self, top_k=None):
        sorted_values = [
            k for k, v in sorted(self._values.items(), key=lambda item: item[1])
        ]
        if top_k is None:
            return sorted_values
        else:
            return sorted_values[:top_k]

    @property
    def n_samples(self):
        return self._count

    @property
    def total_values(self):
        return len(self._values)

    def __repr__(self):
        return "stat(type=categorical, top-5-values=%s, #values=%d, n=%d)" % (
            self.values_top_k(5),
            self.total_values,
            self.n_samples,
        )


class Statistics(object):

    def __init__(self):
        self._stats = dict()

    def update(self, name, values):
        if np.issubdtype(values.dtype, np.number):
            if name not in self._stats:
                self._stats[name] = NumericalStat()
            self._stats[name].update(values)
        else:
            if name not in self._stats:
                self._stats[name] = CategoricalStat()
            self._stats[name].update(values)

    @property
    def stats(self):
        return self._stats

    def save_to_file(self, filepath):
        os.makedirs(os.path.dirname(filepath), exist_ok=True)
        with open(filepath, "wb") as f:
            pickle.dump(self._stats, f)

    def load_from_file(self, filepath):
        with open(filepath, "rb") as f:
            self._stats = pickle.load(f)

    def load_from_textfile(self, filepath, threshold = 0):
        vocab = defaultdict(list)
        with open(filepath, "rt", encoding='utf-8') as f:
            for line in f.readlines():
                fea_cat, fea_value, cnt = line.strip().split("\t")
                if int(cnt) > threshold:
                    vocab[fea_cat].append(fea_value)
        for name in vocab:
            self.update(name, np.array(vocab[name]))

    def __repr__(self):
        return '\n'.join(["%s: %s" % (name, stat) for name, stat in self.stats.items()])


import abc

class BaseStatisticsGen(metaclass=abc.ABCMeta):
    """Base class for a statistics generator component.

    All subclasses of BaseStatisticsGen must override `run` method to generate
    feauture statistics.
    """

    @abc.abstractmethod
    def run(self) -> Statistics:
        """Generate feature statistics.

        :return: Feature statistics results."""
        pass

from collections import defaultdict
class DatasetStatisticsGen(BaseStatisticsGen):

    def __init__(self, iterator, num_batches=None):
        self.iterator = iterator
        self._num_batches = num_batches

    def run(self) -> Statistics:
        sess = tf.Session()
        self.iterator.initializer.run(session = sess)
        #self._dataset.init(sess)
        statistics = Statistics()
        n = 0
        while True:
            try:
                batch = sess.run(self.iterator.get_next())
            except tf.errors.OutOfRangeError:
                break
            #print(batch)
            for name, values in batch.items():
                statistics.update(name, values)
            n += 1
            if n % 1000 == 0:
                print("Statistics collected for %d batches ..." % n)
            if self._num_batches is not None and n >= self._num_batches:
                break
        sess.close()
        print("Statistics collected for %d batches in total." % n)
        return statistics

data_statistic = DatasetStatisticsGen(iterator)
statistic = data_statistic.run()

In [ ]:
len(statistic.stats["movie_id"].values_top_k(None))

In [ ]:
#tf.reset_default_graph()
from typing import Callable

class BaseTransform(metaclass=abc.ABCMeta):

    
    @property
    def transform_fn(self) -> Callable[[tf.train.Example], tf.train.Example]:
        """Returns the transform function.

        :return: A function to transform `tf.Example`."""
        return self._transform_fn

    @abc.abstractmethod
    def _transform_fn(self, example: tf.train.Example) -> tf.train.Example:
        """Transforms `tf.Example`.

        :param example: The input `tf.Example` to be transformed.
        :return: The transformed `tf.Example` as output."""
        return example

map_num_oov_buckets = {}
map_top_k_to_select = {}
map_shared_embedding = {"similar_movies" : "movie_id", "hist_movie_id" : "movie_id", "neighbours":"user_id", "user_id_offline_fixed":"user_id_offline", "user_tags":"movie_tags", "history_similar_movies":"movie_id", "tag_neighbours":"user_id", "tag_similar_movie":"movie_id"}

class CategoricalTransform(BaseTransform):

    def __init__(self,
                 statistics,
                 feature_names,
                 embed_size=32,
                 default_num_oov_buckets=1,
                 map_num_oov_buckets={},
                 map_top_k_to_select={},
                 map_shared_embedding={},
                 scope_name="categorical_transform"):
        self._statistics = statistics
        self._feature_names = feature_names
        self._default_num_oov_buckets = default_num_oov_buckets
        self._map_num_oov_buckets = map_num_oov_buckets
        self._map_top_k_to_select = map_top_k_to_select
        self._map_shared_embedding = map_shared_embedding
        self._embed_size = embed_size
        self._scope_name = scope_name
        self._hash_tables, self.hash_sizes = self._create_hash_tables()
        self._embedding_tables = self._create_embedding_tables(self.hash_sizes)

    def _transform_fn(self, example):
        example = self._hash_lookup(example)
        example = self._embedding_lookup(example)
        return example

    def _hash_lookup(self, example):
        for fea_name in self._feature_names:
            # keep the "user_id_offline_fixed" unchanged
            if fea_name == "user_id_offline_fixed":
                continue
            if fea_name not in self._map_shared_embedding: 
                example[fea_name] = self._hash_tables[fea_name].lookup(
                    example[fea_name]
                )
            else:
                shared_fea_name = self._map_shared_embedding[fea_name]
                example[fea_name] = self._hash_tables[shared_fea_name].lookup(
                    example[fea_name]
                )
        return example

    def _embedding_lookup(self, example):
        for fea_name in self._feature_names:
            # keep the "user_id_offline_fixed" unchanged
            if fea_name == "user_id_offline_fixed":
                continue
            if fea_name not in self._map_shared_embedding:
                example[fea_name] = tf.nn.embedding_lookup(
                    self._embedding_tables[fea_name], example[fea_name]
                )
            else:
                shared_fea_name = self._map_shared_embedding[fea_name]
                example[fea_name] = tf.nn.embedding_lookup(
                    self._embedding_tables[shared_fea_name], example[fea_name]
                )
        return example
    
    def _embedding_modify_fea(self, example, fea_name, value):

        if fea_name not in self._map_shared_embedding:
        #print('self.uid_offlines', self.uid_offlines)
        #print('example[fea_name]', example[fea_name])
            #index_ = example[fea_name]
            print("changed fea_name:", fea_name)
            index_ = tf.transpose([example[fea_name]], perm=[1, 0])
            print("changed_index_", index_)
            print("original self._embedding_tables[fea_name]", self._embedding_tables[fea_name])
            self._embedding_tables[fea_name] = tf.scatter_nd_update(self._embedding_tables[fea_name], index_, value)
            print("value", value)
            print("modified self._embedding_tables[fea_name]", self._embedding_tables[fea_name])
            #select = self._embedding_tables[fea_name][example[fea_name], :]
            #select.assign(value)
            
        else:
            shared_fea_name = self._map_shared_embedding[fea_name]
            tf.assign(self._embedding_tables[shared_fea_name][example[shared_fea_name]], value)
    
    def _hash_lookup_fea(self, example, fea_name):
        if fea_name not in self._map_shared_embedding: 
                example[fea_name] = self._hash_tables[fea_name].lookup(
                    example[fea_name]
                )
        else:
            # find the hash_id of the user_id_offline by looking up the hash value of user_id_offline_fixed  
            shared_fea_name = self._map_shared_embedding[fea_name]            
            print("fea_name", example[fea_name])
            print("shared_fea_name", example[shared_fea_name])
            
            example[shared_fea_name] = self._hash_tables[shared_fea_name].lookup(
                example[fea_name]
            )
        return example
        
    # modi_fea = 'user_id_offline'
    def modify_embedding(self, example, value, modi_fea = 'user_id_offline_fixed'):
        example = self._hash_lookup_fea(example, modi_fea)
        modi_fea = 'user_id_offline'
        self._embedding_modify_fea(example, modi_fea, value)
        

    def _create_hash_tables(self):
        hash_tables = {}
        hash_sizes = {}
        for fea_name in self._feature_names:
            if fea_name in self._map_shared_embedding:
                assert self._map_shared_embedding[fea_name] in self._feature_names
            else:
                num_oov_buckets = (
                    self._map_num_oov_buckets[fea_name]
                    if fea_name in self._map_num_oov_buckets
                    else self._default_num_oov_buckets
                )
                top_k = (
                    self._map_top_k_to_select[fea_name]
                    if fea_name in self._map_top_k_to_select
                    else None
                )
                vocab = []
                print("fea name:",fea_name)
                
                if fea_name in self._statistics.stats:
                    vocab = self._statistics.stats[fea_name].values_top_k(top_k)
                
                else:
                    print(
                        "WARNING: feature [%s] not found in statistics, use empty."
                        % fea_name
                    )
                #print("vocab:",vocab)
                hash_tables[fea_name] = tf.contrib.lookup.index_table_from_tensor(
                    mapping=tf.constant(vocab), num_oov_buckets=num_oov_buckets
                )
                hash_sizes[fea_name] = len(vocab) + num_oov_buckets
        return hash_tables, hash_sizes

    def _create_embedding_tables(self, hash_sizes):
        embedding_tables = {}
        with tf.variable_scope(self._scope_name, reuse=tf.AUTO_REUSE):
            for fea_name in self._feature_names:
                if fea_name in self._map_shared_embedding:
                    assert self._map_shared_embedding[fea_name] in self._feature_names
                else:
                    embedding_tables[fea_name] = tf.get_variable(
                        fea_name + "_embed", [hash_sizes[fea_name], self._embed_size]
                    )
        return embedding_tables

#list(train_dataset.keys())
cate_trans = CategoricalTransform(statistics = statistic, feature_names = ['user_tags', 'movie_tags','user_id','user_id_offline','user_id_offline_fixed','movie_id','hist_movie_id','gender','age','occupation', 'neighbours', "similar_movies",'history_similar_movies','tag_neighbours','tag_similar_movie'], map_num_oov_buckets=map_num_oov_buckets, map_top_k_to_select=map_num_oov_buckets, map_shared_embedding = map_shared_embedding)
#cate_trans._transform_fn(iterator.get_next())

In [ ]:
#tf.reset_default_graph()
from sklearn.metrics import roc_auc_score
class BaseNetwork(metaclass=abc.ABCMeta):
    """Base class for a neural network component.

    All subclasses of BaseNetwork must override _train_fn, _eval_fn and _serve_fn
    methods. _train_fn builds the training graph and returns a loss `tf.Tensor`.
    _eval_fn builds the evaluation graph and outputs a inference `tf.Tensor`.
    _serve_fn might be the same to _eval_fn or do additional graph surgery for
    efficient online serving.
    """

    @property
    def train_fn(self):
        """Returns a function to build training graph.

        :return: A function to build training graph (loss as output)."""
        return self._train_fn

    @property
    def eval_fn(self):
        """Returns a function to build inference graph.

        :return: A function to build inference graph (inference result as output)."""
        return self._eval_fn

    @property
    def serve_fn(self):
        """Returns a function to build serving graph.

        :return: A function to build serving graph (inference result as output)."""
        return self._serve_fn

    @abc.abstractmethod
    def _train_fn(self, example: tf.train.Example) -> tf.Tensor:
        """Build training graph.

        :param example: The `tf.Example` used as graph input.
        :return: A loss `tf.Tensor`."""
        pass

    @property
    def serve_inputs(self):
        """Returns a function to offer inputs for serving.

        :return: A function for offering inputs.
        """
        return self._get_serve_inputs()

    def _tile_tensor_with_batch_size(self, tensor, batch_size):
        """Tile tensor for optimized model serving.

        Tile the tensor to [batch_size, 1] if its first dimension is 1.
        This is used for optimized model serving,
        which has squeezed user/context feature inputs (batch_size = 1)
        and unsqueezed item feature inputs (batch_size > 1).

        :param tensor: Input tensor.
        :param batch_size: Target batch size.
        :return: Tiled tensor with first dimension equal to `batch_size`.
        """
        dims = len(tensor.get_shape().as_list())
        shape = [1] * dims
        shape[0] = batch_size
        output = tf.cond(tf.equal(tf.shape(tensor)[0], 1),
                         lambda: tf.tile(tensor, shape),
                         lambda: tensor)
        return output

    def _tile_tensors_with_batch_size(self, tensors, batch_size):
        """Tile tensors for optimized model serving.

        For every tensor of tensors,
        do _tile_tensor_with_batch_size(tensor, batch_size).

        :param tensors: Input tensors list.
        :param batch_size: Target batch size.
        :return: Tiled tensors with first dimension of
        every tensor equal to `batch_size`.
        """
        return [
            self._tile_tensor_with_batch_size(tensor, batch_size)
            for tensor in tensors
        ]

    def _get_batch_size(self, example):
        """Get the batch size.

        Get the batch size by traversing all features' shape in example.

        :param example: Dict of string->tensor.
        :return: A tensor of batch_size.
        """
        batch_size_tensors = [tf.shape(tensor)[0] for key, tensor in example.items()
                              if key in self.serve_inputs]
        batch_size = tf.reduce_max(batch_size_tensors)
        return batch_size

import tensorflow.contrib.slim as slim

class MlpNetwork(BaseNetwork):

    def __init__(self,
                 train,
                 cate_trans,
                 item_features,
                 categorical_features,
                 numerical_features,
                 multivalue_features,
                 loss,
                 hidden_sizes=[1024, 512, 128],
                 scope_name="mlp_network", ptype = "mean"):
        self._train = train
        self.item_features = item_features
        self.cate_trans = cate_trans
        self._categorical_features = categorical_features
        self._numerical_features = numerical_features
        self._multivalue_features = multivalue_features
        self._loss = loss
        self._hidden_sizes = hidden_sizes
        self._scope_name = scope_name
        self._ptype = ptype

    def _train_fn(self, example):
        with tf.variable_scope(self._scope_name, reuse=tf.AUTO_REUSE):
            logits = self._build_graph(example)
            loss = self._loss.loss_fn(logits, example)
            labels = tf.expand_dims(tf.to_int32(tf.strings.to_number(example['label'])), 1, name=None) 
            return loss, logits, labels

    def _eval_fn(self, example):
        with tf.variable_scope(self._scope_name, reuse=tf.AUTO_REUSE):
            logits = self._build_graph(example)
            outputs = tf.sigmoid(logits)
            return outputs

    def _serve_fn(self, example):
        return self._eval_fn(example)
    
    
    def _build_multivalue_part(self, inputs):
        def pooling(vals, ptype, fea_name):
            if ptype == "mean":
                return tf.reduce_mean(vals, axis=1)
            elif ptype == "sum":
                return tf.reduce_sum(vals, axis=1)
            elif ptype == "fc":
                return tf.squeeze(slim.fully_connected(
                    tf.transpose(vals, [0, 2, 1]),
                    1,
                    scope="multivalue_fc_%s" % fea_name))

        outputs = [pooling(inputs[name], self._ptype, name) 
            for name in self._multivalue_features]
        return outputs
    
    
    def _build_item_part(self, inputs):
        def pooling(vals, ptype, fea_name):
            if ptype == "mean":
                return tf.reduce_mean(vals, axis=1)
            elif ptype == "sum":
                return tf.reduce_sum(vals, axis=1)
            elif ptype == "fc":
                return tf.squeeze(slim.fully_connected(
                    tf.transpose(vals, [0, 2, 1]),
                    1,
                    scope="multivalue_fc_%s" % fea_name))

        outputs = [pooling(inputs[name], self._ptype, name) 
            for name in self.item_features]
        return outputs
    
    def build_HAN(self, inputs, feature, target_feature, attention_enhance, trans_embed = 16):
        #user HAN enhance
        if attention_enhance:
            neighbours = inputs[feature]
            
            trans_W = tf.get_variable("trans_W", [inputs[target_feature].shape[-1], trans_embed], trainable = True)
            
            q_1 = tf.get_variable("q_1", [trans_embed, 1], trainable = True)
            q_2 = tf.get_variable("q_2", [trans_embed, 1], trainable = True)
            expand_self = tf.expand_dims(inputs[target_feature], 1)
            
            neighbours_ = tf.matmul(neighbours, trans_W)
            expand_self_ = tf.matmul(expand_self, trans_W)
            
            f_1 = tf.matmul(neighbours_, q_1)
            self_node = tf.matmul(expand_self_, q_2)

            logits = self_node + f_1
            logits = tf.squeeze(logits, [-1])
            coefs = tf.nn.softmax(tf.nn.leaky_relu(logits))
            coefs_ = tf.expand_dims(coefs, axis = 1)
            vals = tf.matmul(coefs_, neighbours_)
            vals = tf.squeeze(vals, axis = 1)

        else:
            vals = tf.reduce_mean(inputs[feature], axis = 1)
        #ret = tf.contrib.layers.bias_add(vals)
        return vals
        

    def _build_graph(self, inputs):
        
        user_enhance = self.build_HAN(inputs, 'neighbours','user_id', False)
        item_enhance = self.build_HAN(inputs, 'similar_movies', 'movie_id', False)
        
        item_tags = tf.concat(self._build_item_part(inputs), axis = 1, name = "concat_0")
        multivalue_features_ = tf.concat(self._build_multivalue_part(inputs), axis = 1, name = "concat_1")
        
        categorical_part = tf.concat(
            [inputs[name] for name in self._categorical_features],
            axis=1,name = "concat__2"
        )
        
        user_id_embed = inputs['user_id']
        if self._train:
            user_id_embed = user_id_embed
        else:
            user_id_embed = user_id_embed

        with tf.control_dependencies([item_tags, inputs['movie_id'], item_enhance]):
            item_part = tf.concat([item_tags, inputs['movie_id'], item_enhance], axis = 1, name = "concat_2")
        with tf.control_dependencies([multivalue_features_, user_enhance]):
            user_part = tf.concat([multivalue_features_, user_enhance], axis = 1, name = "concat_3")
        
        #with tf.control_dependencies([user_id_embed, history_movies,user_enhance]):
        #    user_part = tf.concat([user_id_embed, history_movies,user_enhance], axis = 1, name = "concat_3")

        with tf.control_dependencies([user_part, item_part]): 
            hidden = tf.concat([user_part, item_part], axis=1,name = "concat_4")

        #print("multi_value_part.shape", multi_value_part.shape)
        #print("categorical_part.shape", categorical_part.shape)
        #print("hidden.shape", hidden.shape)
        
        hidden_1 = tf.compat.v1.layers.dense(hidden, 32, activation=tf.nn.leaky_relu, use_bias=True, name="fc_0", reuse=tf.AUTO_REUSE)
        hidden_2 = tf.compat.v1.layers.dense(hidden_1, 16, activation=tf.nn.leaky_relu, use_bias=True, name="fc_1", reuse=tf.AUTO_REUSE)
        outputs = tf.compat.v1.layers.dense(hidden_2, 1, activation=None, use_bias=True, name="logit", reuse=tf.AUTO_REUSE)
        
        return outputs

class BaseLoss(metaclass=abc.ABCMeta):
    """Base class for a loss function component.

    All subclasses of BaseLoss must override `loss_fn` method.
    """

    @abc.abstractmethod
    def loss_fn(self, logits: tf.Tensor, examples: tf.train.Example) -> tf.Tensor:
        """Build loss function.

        :param logits: The input logits to build the loss function.
        :param examples: The input `tf.Example` from which to abtain
            all required data fields.
        :return: A loss `tf.Tensor`. """
        pass


class CrossEntropyLoss(BaseLoss):

    def __init__(self, label_name):
        self._label_name = label_name

    def loss_fn(self, logits, examples):
        print(examples[self._label_name])
        labels = tf.expand_dims(
            tf.to_float(tf.strings.to_number(examples[self._label_name])), 1, name=None)    
        return self._cross_entropy_loss(logits, labels)

    def _cross_entropy_loss(self, logits, labels):
        
        sample_loss = tf.nn.sigmoid_cross_entropy_with_logits(
            logits=logits, labels=labels
        )
        
        avg_loss = tf.reduce_mean(sample_loss)
        return avg_loss
import json
    

class Trainer(object):
    
    def __init__(self,
                 train_iterator,
                 cate_trans,
                 test_iterator,
                 transform_functions,
                 train_fn,
                 test_fn,
                 validate_steps,
                 log_steps,
                 learning_rate,
                 train_epochs=1,
                 evaluator=None,
                 save_checkpoints_dir=None,
                 restore_checkpoint_path=None,
                 validate_at_start=False,
                 tensorboard_logdir="./"):
        
        #self.neighbours = neighbours
        self.train_iterator = train_iterator
        self.cate_trans = cate_trans
        self.test_iterator = test_iterator
        self._transform_functions = transform_functions
        self._train_fn = train_fn
        self._test_fn = test_fn
        self._train_epochs = train_epochs
        self._save_checkpoints_dir = save_checkpoints_dir
        self._restore_checkpoint_path = restore_checkpoint_path
        self._validate_steps = validate_steps
        self._log_steps = log_steps
        self._learning_rate = learning_rate
        self._validate_at_start = validate_at_start
        self._evaluator = evaluator
        self.total_loss = []
        self.train_loss, self.train_logits, self.train_example, self.test_loss, self.test_logits, self.test_example, self._train_op = self._build_train_graph()
        #self._valid_logger = ValidateLogger(tensorboard_logdir)
        self._train_logger = TrainLogger(self._log_steps, tensorboard_logdir)

    def run(self, sess=None):
        self.total_loss = 0
        if sess is None:
            self._sess = self._create_and_init_session()
        else:
            self._sess = sess

        self._train_loop()
        
        self.save_embeddings()
        
        if sess is None:
            self._sess.close()
            
    def save_embeddings(self):
        user_ids = self._sess.run(self.cate_trans._embedding_tables['user_id']).tolist()
        item_ids = self._sess.run(self.cate_trans._embedding_tables['movie_id']).tolist()
        
        dict_ids = {
            "user_ids":user_ids,
            "item_ids":item_ids
        }
        
        with open('ids.json', 'w') as outfile:
            json.dump(dict_ids, outfile)

    def _create_and_init_session(self):
        NUM_THREADS = 200
        sess = tf.Session(config=tf.ConfigProto(
  intra_op_parallelism_threads=NUM_THREADS))
        tf.global_variables_initializer().run(session=sess)
        tf.tables_initializer().run(session=sess)
        return sess

    def _build_train_graph(self):
        trainable_params = tf.trainable_variables()
        print("trainable_params", trainable_params)
        trainer = tf.train.AdamOptimizer(learning_rate=self._learning_rate)
        transform_fn = self._join_pipeline(self._transform_functions)
        train_loss, train_logits, train_example = self._train_fn(transform_fn(self.train_iterator.get_next()))
        test_loss, test_logits, test_example = self._test_fn(transform_fn(self.test_iterator.get_next()))
        #self.neighbours = transform_fn(self._iterator.get_next())['neighbours']
        grads = tf.gradients(train_loss, trainable_params)
        train_op = trainer.apply_gradients(list(zip(grads, trainable_params)))
        return train_loss, train_logits, train_example, test_loss, test_logits, test_example, train_op

    def _train_loop(self):
        step = 0
        self.total_loss = []
        epoch = 0
        while epoch < self._train_epochs:
            self.train_iterator.initializer.run(session = self._sess)
            total_logits = []
            total_labels = []
            continue_ = True
            while True:
                success = self._train_step(epoch, step, total_logits, total_labels)
                if not success:
                    auc_ = self.cal_auc(total_logits, total_labels)
                    print("auc_", auc_)
                    self._test_loop()
                    if len(self.total_loss) > 2:
                        if self.total_loss[-1] < self.total_loss[-2]:
                            continue_ = False
                            print("Overfitting")
                    break
                step += 1
            if not continue_:
                break
            epoch += 1
            if epoch % 50 == 0:
                self.save_embeddings()
                #if step % self._validate_steps == 0:
                #    self._validate(epoch=epoch, step=step)
            #self._save_checkpoint(epoch + 1)

    def _train_step(self, epoch, step, total_logits, total_labels):
        try:
            t_start = time.time()
            train_loss, train_logits, train_example = self._sess.run([self.train_loss, self.train_logits, self.train_example, self._train_op])[0:3]
            total_logits.append(train_logits)
            total_labels.append(train_example)
            
            t_end = time.time()
            self._train_logger.log_info(loss=train_loss,
                                        time=t_end - t_start,
                                        size=10,
                                        epoch=epoch,
                                        step=step + 1)
            return True
        except tf.errors.OutOfRangeError:
            return False
        
    def cal_auc(self, logits, example):
        logits = np.array(logits).reshape(-1, 1)
        example = np.array(example).reshape(-1, 1)
        
        #logits = np.concatenate(np.array(logits), axis = 0)
        #example = np.concatenate(np.array(example), axis = 0)
        
        auc = roc_auc_score(y_true=example, y_score=logits)
        return auc
            
    def _test_loop(self):
        self.test_iterator.initializer.run(session = self._sess)
        total_logits = []
        total_labels = []
        total_loss = []
        while True:
            #print("why")
            success = self._test_step(total_logits, total_labels, total_loss)
            if success == -1:
                auc_test = self.cal_auc(total_logits, total_labels)
                print("test auc", auc_test)
                self.total_loss.append(auc_test)
                break
    
    def _test_step(self,total_logits, total_labels, total_loss):
        try:
            test_loss, test_logits, test_example = self._sess.run([self.test_loss, self.test_logits, self.test_example])
            total_logits.append(test_logits)
            total_labels.append(test_example)
            total_loss.append(test_loss)
            return test_loss
        except tf.errors.OutOfRangeError:
            return -1

    def _save_checkpoint(self, step, prefix="ckpt_epoch"):
        if self._save_checkpoints_dir:
            checkpoint_saver = tf.train.Saver(max_to_keep=None)
            checkpoint_path = os.path.join(self._save_checkpoints_dir, prefix)
            checkpoint_saver.save(self._sess, checkpoint_path, global_step=step)

    def _restore_checkpoint(self):
        #print("model path",self._restore_checkpoint_path)
        #and  os.listdir(self._restore_checkpoint_path)
        if os.path.exists(self._restore_checkpoint_path):
            checkpoint_saver = tf.train.Saver(max_to_keep=None)
            checkpoint_saver.restore(self._sess, self._restore_checkpoint_path)

    def _validate(self, epoch, step):
        if self._evaluator is not None:
            eval_results = self._evaluator.run(sess=self._sess)
            self._valid_logger.log_info(eval_results, epoch=epoch, step=step)

    def _join_pipeline(self, map_functions):
        
        def joined_map_fn(example):
            for map_fn in map_functions:
                example = map_fn(example)
            return example

        return joined_map_fn

cate_features = ['age', 'gender', 'occupation']
multivalue_features = ['hist_movie_id']
item_features = ['movie_tags']
nume_features = []

train_network = MlpNetwork(
    train = True,
    cate_trans = cate_trans,
    item_features = item_features,
    categorical_features=cate_features,
    numerical_features=nume_features,
    multivalue_features=multivalue_features,
    loss=CrossEntropyLoss(label_name = 'label')
)

test_network = MlpNetwork(
    train = False,
    cate_trans = cate_trans,
    item_features = item_features,
    categorical_features=cate_features,
    numerical_features=nume_features,
    multivalue_features=multivalue_features,
    loss=CrossEntropyLoss(label_name = 'label')
)

class TrainLogger(object):

    def __init__(self, log_steps, tensorboard_logdir=None):
        self._log_steps = log_steps
        self.last_epoch = 0
        self.total = 0
        self._tensorboard_writer = None
        if tensorboard_logdir:
            self._tensorboard_writer = tf.summary.FileWriter(tensorboard_logdir)
        self._cleanup()

    def log_info(self, loss, time, size, epoch, step):
        self._total_loss += loss
        self._total_time += time
        self._total_size += size
        self._total_steps += 1

        if self._total_steps >= self._log_steps:
            avg_loss = self._total_loss / self._total_steps
            fps = self._total_size / float(self._total_time)
            self._log_to_console(avg_loss, self._total_time, fps, epoch, step)
            self._log_to_tensorboard(loss, fps, step)
            self._cleanup()

    def _log_to_console(self, loss, time, fps, epoch, step):
        if self.last_epoch == epoch:
            self.total += loss
        else:
            print("[Train] Epoch: %d\tLoss: %.5f\t", epoch, self.total/1000)
            self.total = 0
        print(
            "[Train] Epoch: %d\tStep: %d\tLoss: %.5f\tTime: %.2f\tFPS: %d"
            % (epoch, step, loss, time, fps)
        )
        self.last_epoch = epoch

    def _log_to_tensorboard(self, loss, fps, step):
        if self._tensorboard_writer:
            summary = tf.Summary(
                value=[tf.Summary.Value(tag="train_loss", simple_value=loss),
                       tf.Summary.Value(tag="train_fps", simple_value=fps)]
            )
            self._tensorboard_writer.add_summary(summary, step)
            self._tensorboard_writer.flush()

    def _cleanup(self):
        self._total_loss = 0
        self._total_time = 0
        self._total_instance = 0
        self._total_size = 0
        self._total_steps = 0


class ValidateLogger(object):
    def __init__(self, tensorboard_logdir=None):
        self._tensorboard_writer = None
        if tensorboard_logdir:
            self._tensorboard_writer = tf.summary.FileWriter(tensorboard_logdir)

    def log_info(self, metric_results, epoch, step):
        self._log_to_console(metric_results, epoch, step)
        self._log_to_tensorboard(metric_results, step)

    def _log_to_console(self, metric_results, epoch, step):
        epoch_total = 0
        results_str = "\t".join(
            ["%s: %s" % (metric_name, metric_result)
             for metric_name, metric_result in metric_results.items()]
        )
        print("[Validate] Epoch: %d\tStep: %d\t%s" % (epoch, step, results_str))

    def _log_to_tensorboard(self, metric_results, step):
        if self._tensorboard_writer:
            summary = tf.Summary(
                value=[tf.Summary.Value(tag=("valid_%s" % metric_name),
                                        simple_value=metric_result.result)
                       for metric_name, metric_result in metric_results.items()]
            )
            self._tensorboard_writer.add_summary(summary, step)
            self._tensorboard_writer.flush()

trainer = Trainer(

    train_iterator=iterator,
    cate_trans = cate_trans,
    test_iterator = test_iterator,
    transform_functions=[cate_trans.transform_fn],
    train_fn=train_network.train_fn,
    test_fn = test_network.train_fn,
    validate_steps=5,
    log_steps=10,
    learning_rate=0.005,
    train_epochs=500,
    save_checkpoints_dir=10,
    restore_checkpoint_path=20,
    tensorboard_logdir=10,  
)

In [ ]:
#auc_ 0.9974448867513553
#test auc 0.9859417127610063
#test auc 0.990621
#auc_ 0.9970889560539397
#test auc 0.9887362768997808
#auc_ 0.9998272457738399
#test auc 0.9964203584405279

#auc_ 0.999725004116234
#test auc 0.9847685588551847
#user item combined
#auc_ 0.9999014938477996
#test auc 0.9888321861816355

0.9765
(0.983025 + 0.9829226638492917 + 0.9813233946314516)/3
# - 0.983025 + 0.96834 - 0.9829226638492917 + 0.9734312800631626 - 0.9813233946314516)/6 
#(0.9840043662397732/ 0.9843278173647924/ 0.986869226965459）- 0.9764813519305374 / 0.981874454044821/0.9843495799561665

In [ ]:
trainer.run()

In [ ]:
sparse_users = all_usrs[all_usrs < 30]
sparse_user_df = merged_df[merged_df.user_id.isin(sparse_users.index)]
sparse_user_df.to_csv("sparse_merged_df.csv")

In [ ]:
len(statistic.stats["user_id"].values_top_k(None))

In [ ]:
len(interest_dict)

In [ ]:
#print(len(merged_df.movie_id.unique()))
user_ids_ = statistic.stats["movie_id"].values_top_k(None)

In [ ]:
#han_user
import json

with open('ids.json') as json_file:
    data = json.load(json_file)

In [ ]:
user_id_dict = {}
user_ids = data['item_ids'][:-1]
for i in range(len(user_ids)):
    user_id_dict[user_ids_[i]] = user_ids[i]

In [ ]:
user_id_dict

In [ ]:
len(merged_df.movie_id.unique())

In [ ]:
extract_mdf = movie_df[movie_df.movie_id.isin(merged_df.movie_id.unique())]

In [ ]:
sorted_dict = sorted([(int(a),b) for a,b in user_id_dict.items()], key = lambda x : x[0])

In [ ]:
#user_df = pd.DataFrame(data={
#    "tags":list(interest_dict.values()),
#    "embed":[x[1] for x in sorted_dict],
#},index=[x[0] for x in sorted_dict])

In [ ]:
extract_mdf['item_embed'] = [x[1] for x in sorted_dict]

In [ ]:
extract_mdf

In [ ]:
user_hist_dict = {}
for u,u_hist in merged_df.groupby('movie_id'):
    user_hist_dict[u] = u_hist.movie_id.values

In [ ]:
user_hist_list = [a for a in list(user_hist_dict.values())]

In [ ]:
user_df['user_hist'] = user_hist_list
user_df['hist_len'] = user_df.user_hist.apply(lambda x:len(x))

In [ ]:
user_df

In [ ]:
new_column = [x[1] for x in sorted_dict]

In [ ]:
user_df[user_df.index == 1]['embed'].values[0]

In [ ]:
def sort_lines(df, id_, name, user, advanced, version):
    target_line = df[df.movie_id == id_][name].values[0]
    simila = []
    for i in range(df.shape[0]):
        line_ = df.iloc[i][name]
        simila.append((df.iloc[i].movie_id, np.dot(target_line, line_)/(np.linalg.norm(target_line)*np.linalg.norm(line_))))
    #df['sort'] = simila
    #df.sort_values(by = 'sort', inplace = True, ascending = False)
    simila = sorted(simila, key = lambda x:x[1], reverse = True)
    return [x[0] for x in simila[:5]]
    """
    advan = "normal"
    if advanced:
        advan = "advan"
    else:
        advan = "normal"
    if user:
        df.to_csv("user_{}_similarity_{}_version{}.csv".format(id_, advan, version))
    else:
        df.to_csv("item_{}_similarity_{}_version{}.csv".format(id_, advan, version))
    """

In [ ]:
res = {}
for i in range(extract_mdf.shape[0]):
    m_id = extract_mdf.iloc[i].movie_id
    res[m_id] = sort_lines(extract_mdf, m_id, 'item_embed', False, True, 3)

In [ ]:
#delete the test dataset info to prevent data-leakage
test_index = []
for u_id, hist in merged_df.groupby('user_id'):
    test_index.append(hist.iloc[-1].name)
train_merged_df = merged_df[~merged_df.index.isin(test_index)]

#create a itemcf user_item matrix
max_item = max(train_merged_df.movie_id.unique() + 1)
max_user = max(train_merged_df.user_id.unique() + 1)
co_matrix = np.zeros((max_item,max_user))

print(co_matrix.shape)

for row_num in range(train_merged_df.shape[0]):
    row = train_merged_df.iloc[row_num]
    co_matrix[row.movie_id, row.user_id] = row.rating

def cos_value(a, b):
    return np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))

movie_id_list = train_merged_df.movie_id.value_counts().index
simi_dict = {}
for item in tqdm(movie_id_list):
    temp_list = []
    for other_item in movie_id_list:
        temp_list.append((other_item, cos_value(co_matrix[item], co_matrix[other_item])))
    temp_list = sorted(temp_list, key = lambda x:x[1], reverse = True)
    temp_list = [x[0] for x in temp_list[:10]]
    simi_dict[item] = temp_list

In [ ]:
simi_dict

In [ ]:
movie_1 = 741
movie_2 = 3858
print(len(set(merged_df[merged_df.movie_id == movie_1].user_id.values)))
print(len(set(merged_df[merged_df.movie_id == movie_2].user_id.values)))
set(merged_df[merged_df.movie_id == movie_1].user_id.values).intersection(set(merged_df[merged_df.movie_id == movie_2].user_id.values))

In [ ]:
merged_df.movie_id.value_counts().index[::-1][:100]

In [ ]:
movie_df[movie_df.movie_id.isin(simi_dict[301])]

In [ ]:
new_res = {}
for i in res:
    new_res[str(i)] = str(list(res[i]))

In [ ]:
len(extract_mdf.movie_id.unique())

In [ ]:
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

with open('similar_items.json', 'w') as outfile:
    json.dump(simi_dict, outfile, cls=NumpyEncoder) 

In [ ]:
sort_lines(extract_mdf, 1, 'item_embed', False, True, 3)

In [ ]:
#print(len(statistic.stats["user_id"].values_top_k(None)))
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
res = pca.fit_transform(data['item_ids'])

plt.figure()
#lw = 2

#for color, i, target_name in zip(colors, [0, 1, 2], target_names):
plt.scatter(res[:, 0], res[:, 1],alpha=0.1)


In [ ]:
res

In [ ]:
trainer.run()

In [ ]:
trainer.run()

In [ ]:
trainer.run()

In [ ]:
trainer.run()

In [ ]:
with tf.Session() as sess:
    #initilize the iterator
    iterator.initializer.run()
    while True:
        try:
            ne = iterator.get_next()
            read_features = dict()
            for key in train_dataset.keys():
                if key == 'hist_movie_id' or key == 'neighbours' :
                    read_features[key] = tf.FixedLenSequenceFeature([],dtype=tf.string,allow_missing=True)
                else:
                    read_features[key] = tf.FixedLenFeature([], dtype=tf.string)

            # Extract features from serialized data
            read_data = tf.parse_example(serialized=ne,
                                                features=read_features)

            for k, v in read_data.items():
                read_data[k] = tf.identity(v, name=k)
            print(read_data['user_id'].eval())
        except tf.errors.OutOfRangeError:
            break

In [ ]:
dataset = tf.data.TFRecordDataset('train.tfrecord')
dataset = dataset.batch(1000, True)
#dataset = _dataset_map(dataset, [_transform])
iterator = dataset.make_initializable_iterator()
with tf.Session() as sess:
    #initilize the iterator
    iterator.initializer.run()
    while True:
        try:
            ne = iterator.get_next()
            read_features = dict()
            for key in ['user_tags', 'movie_tags', 'user_id_offline', 'user_id_offline_fixed', 'age', 'gender', 'hist_movie_id', 'label', 'movie_id', 'neighbours', 'occupation', 'user_id']:
                if key == 'hist_movie_id' or key == 'neighbours' or key == 'user_tags' or key == 'movie_tags':
                    read_features[key] = tf.FixedLenSequenceFeature([],dtype=tf.string,allow_missing=True)
                else:
                    read_features[key] = tf.FixedLenFeature([], dtype=tf.string)

            # Extract features from serialized data
            read_data = tf.parse_example(serialized=ne,
                                                features=read_features)

            for k, v in read_data.items():
                read_data[k] = tf.identity(v, name=k)
            print(read_data['movie_tags'].eval())
        except tf.errors.OutOfRangeError:
            break

In [ ]:
data_handler.create_next_example

In [ ]:
gen_data = get_single_data(train_dataset, data_fea)

In [ ]:
next(gen_data)

In [ ]:
type2movie = {}
for i in range(movie_df.shape[0]):
    movie = movie_df.iloc[i]
    for type_ in movie.Genres:
        if type_ in type2movie:
            type2movie[type_].append(movie.MovieID)
        else:
            type2movie[type_] = [movie.MovieID]

In [ ]:
[(i, len(type2movie[i])) for i in type2movie]

In [ ]:
item2neighbours = {}
for i in range(movie_df.shape[0]):
    movie = movie_df.iloc[i]
    total_neigh = set()
    for type_ in movie.Genres:
        total_neigh = total_neigh.union(set(type2movie[type_]))
    item2neighbours[movie.MovieID] = total_neigh

In [ ]:
nb_nei = [(i, len(item2neighbours[i])) for i in item2neighbours]

In [ ]:
pd.Series([i[1] for i in nb_nei]).hist(bins = 100)

In [ ]:
user_df.UserID = user_df.index

In [ ]:
age2user = {}
occ2user = {}
for i in range(user_df.shape[0]):
    user = user_df.iloc[i]
    if user.Age in age2user:
        age2user[user.Age].append(user.UserID)
    else:
        age2user[user.Age] = [user.UserID]
    if user.Age in occ2user:
        occ2user[user.Occupation].append(user.UserID)
    else:
        occ2user[user.Occupation] = [user.UserID]

In [ ]:
user2neighbours = {}
for i in range(user_df.shape[0]):
    user = user_df.iloc[i]
    total_neigh = set()
    for age_ in [user.Age]:
        total_neigh = total_neigh.union(set(age2user[age_]))
    for occ_ in [user.Occupation]:
        total_neigh = total_neigh.union(set(occ2user[occ_]))
    user2neighbours[user.UserID] = total_neigh

In [ ]:
pd.Series([len(user2neighbours[a]) for a in user2neighbours]).hist(bins = 50)

In [ ]:
user2neighbours = {str(a): str(user2neighbours[a]) for a in user2neighbours}
item2neighbours = {str(b): str(item2neighbours[b]) for b in item2neighbours}

In [ ]:
import json
with open('user2neighbours', 'w') as json_file:
    json.dump(user2neighbours, json_file)
with open('item2neighbours', 'w') as json_file:
    json.dump(item2neighbours, json_file)

In [ ]:
user_df.UserID.max()

In [ ]:
rating_df.UserID = rating_df.UserID.apply(lambda x:x-1)

In [ ]:
movie_df.MovieID

In [ ]:
movie_df_ = pd.read_csv("movies.dat", sep="::", header = None)

In [ ]:
movieID_map = {b:a for (a,b) in movie_df_.iloc[:,0].to_dict().items()}

In [ ]:
rating_df.MovieID = rating_df.MovieID.map(movieID_map)

In [ ]:
rating_df.UserID = rating_df.UserID.apply(lambda x : x - 1)

In [ ]:
sorted_rating = rating_df.sort_values(by = "Timestamp")

In [ ]:
sorted_rating

In [ ]:
y = sorted_rating.Rating
X = sorted_rating.drop(['Rating'], axis = 1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(sorted_rating, test_size=0.33, random_state=42, stratify = sorted_rating["UserID"])


In [ ]:
X_train.to_csv("train_data.csv", header = None, index = None)

In [ ]:
X_test.to_csv("test_data.csv", header = None, index = None)

In [ ]:
len(X_test.UserID.unique())

In [ ]:
X_train.sort_values(by = 'UserID')

In [ ]:
670140/1000

In [ ]:
X_test.sort_values(by = 'UserID')

In [ ]:
rating_train = sorted_rating.iloc[:int(sorted_rating.shape[0] * 2/3) , :] 

In [ ]:
rating_train.to_csv("train_data.csv", header = None, index = None)

In [ ]:
rating_test = sorted_rating.iloc[-int(sorted_rating.shape[0] * 1/3): , :] 

In [ ]:
rating_test.to_csv("test_data.csv", header = None, index = None)

In [ ]:
rating_train

In [ ]:
np.ones((1,300,1)) + np.ones((1,1,300))

In [ ]:
pd.read_csv("train_data.csv", header = None)